In [ ]:
from audio import open_stream, get_audio_data, get_saved_audio
import os
import numpy as np
import pygame
from pygame import Color, surfarray

AUD_DIR = os.path.join('Media', 'Audio')
AUD_FILE = os.path.join(AUD_DIR, 'galaxy rise - drums.wav')

SCREEN_WIDTH = 800
SCREEN_HEIGHT = 800

cell_size = 80 # Number of pixels per cell

cell_width = SCREEN_WIDTH//cell_size
cell_height = SCREEN_HEIGHT//cell_size

display_width = cell_width*cell_size
display_height = cell_height*cell_size

In [ ]:
saved_data = get_saved_audio(AUD_FILE)
if saved_data:
    audio_data, CHUNK, RATE = saved_data
else:
    audio_data, CHUNK, RATE = get_audio_data(AUD_FILE)

In [ ]:
def fft_to_buckets(freq, PSD, buckets):
    """
    Takes the current CHUNK's frequency response and breaks each frequency in to buckets
    - freq: audio files CHUNK of data amplitudes converted in to frequencies
    - PSD: power spectral density of each frequency
    - buckets: a list of frequencies where each freq in the list will create a range between that freq and the previous
    """
    idxs = sorted({np.abs(freq - i).argmin() for i in buckets}) # Get indices of freq from closest frequencies in buckets

    # Average PSD values in between frequencies defined by buckets
    freq_bucket = [PSD[idxs[i]:idxs[i+1]].mean() for i in range(len(idxs)-1)]  + [PSD[idxs[-1]:].mean()] 

    return freq_bucket, idxs

def get_minmax_bucket_freq(audio_data):
    for i, data in enumerate(audio_data):
        n = len(data) 
        fhat = np.fft.fft(data, n)
        PSD = np.abs(fhat * np.conj(fhat) / n) # Power Spectral Density
        freq = (RATE / n) * np.arange(n)

        fb, idxs = fft_to_buckets(freq, PSD, buckets) # Chunk frequencies in to buckets

        if i == 0:
            all_buckets = np.array(fb)
        else:
            all_buckets = np.vstack((all_buckets, fb))
            
    return np.min(all_buckets, axis=0), np.max(all_buckets, axis=0)

min_bucket, max_bucket = get_minmax_bucket_freq(audio_data)

def new_state(freq_buckets, min_bucket, max_bucket):
    state = np.zeros((cell_height, cell_width))
    
    bucket_px_width = int(cell_width/len(freq_buckets))
    
    # Scale buckets
    scaled = (freq_buckets - min_bucket)/(max_bucket - min_bucket)
    multiplier = 2
    scaled = [int(np.round(s*cell_height*multiplier)) for s in scaled for _ in range(bucket_px_width)]
    scaled = [s if s < cell_height else cell_height - 1 for s in scaled]
    
    for i, s in enumerate(scaled):
        state[i, :s] = 1
     
    state = np.flip(state, axis=1)
    return state

def state_to_px(state, px):
    temp = np.where(state == 1, 0, 255)
    for i in range(px.shape[-1]):
        px[:,:,i] = temp
    return px

buckets = [31.25 * 2 ** (n) for n in range(10)]

In [ ]:
stream, wf, CHUNK = open_stream(AUD_FILE)

state = np.zeros((cell_height, cell_width))

pygame.init()

screen = pygame.display.set_mode((display_height, display_width))
pygame.display.set_caption('Threshold Helper')

#create a surface with the size as the array
surface = pygame.Surface((cell_height, cell_width))
surface = pygame.transform.scale(surface, (display_height, display_width)) # Scale Size Up

# Initialize screen to white
px = pygame.surfarray.pixels3d(surface)
px[:,:,:] = np.ones((display_height, display_width, 3))*255 

run = True
cnt = 0
while run:
    for event in pygame.event.get():
        if event.type == pygame.QUIT: 
            run = False
    
    # Scale Screen Size Down to edit pixel array at cell size=1 --------------------------------------------------
    surface = pygame.transform.scale(surface, (cell_height, cell_width))
    px = pygame.surfarray.pixels3d(surface)
      
    data = audio_data[cnt]
    
    stream.write(bytes(data))
    
    n = len(data) 
    fhat = np.fft.fft(data, n)
    PSD = np.abs(fhat * np.conj(fhat) / n) # Power Spectral Density
    freq = (RATE / n) * np.arange(n)
    
    fb, idxs = fft_to_buckets(freq, PSD, buckets) # Chunk frequencies in to buckets
        
    # Iterate State
    state = new_state(fb, min_bucket, max_bucket)
    px = state_to_px(state, px)
    
    # Scale Screen Size Up to display pixel array at cell size=multiplier ----------------------------------------
    surface = pygame.transform.scale(surface, (display_height, display_width))  # Scale Size Up     
    screen.blit(surface, (0, 0)) # Update all pixels on screen object
    
    pygame.display.update()

    cnt += 1
    
pygame.quit()